In [1]:
import cohere 
import pandas as pd
import numpy as np
from tqdm import tqdm

/Users/saurabh/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
api_key = 'aAryunAZIP5LMIXHSuUQ5bIFyiOjx5NkaEm6mlXC'

co = cohere.Client(api_key)

In [3]:
text = """
Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan. 
It stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, Matt Damon, and Michael Caine. 
Set in a dystopian future where humanity is struggling to survive, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for mankind.

Brothers Christopher and Jonathan Nolan wrote the screenplay, which had its origins in a script Jonathan developed in 2007. 
Caltech theoretical physicist and 2017 Nobel laureate in Physics[4] Kip Thorne was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar. 
Cinematographer Hoyte van Hoytema shot it on 35 mm movie film in the Panavision anamorphic format and IMAX 70 mm. 
Principal photography began in late 2013 and took place in Alberta, Iceland, and Los Angeles. 
Interstellar uses extensive practical and miniature effects and the company Double Negative created additional digital effects.

Interstellar premiered on October 26, 2014, in Los Angeles. 
In the United States, it was first released on film stock, expanding to venues using digital projectors. 
The film had a worldwide gross over $677 million (and $773 million with subsequent re-releases), making it the tenth-highest grossing film of 2014. 
It received acclaim for its performances, direction, screenplay, musical score, visual effects, ambition, themes, and emotional weight. 
It has also received praise from many astronomers for its scientific accuracy and portrayal of theoretical astrophysics. Since its premiere, Interstellar gained a cult following,[5] and now is regarded by many sci-fi experts as one of the best science-fiction films of all time.
Interstellar was nominated for five awards at the 87th Academy Awards, winning Best Visual Effects, and received numerous other accolades"""

In [5]:
# splitting

texts = text.split('.')

texts = [t.strip('\n') for t in texts]

texts

['Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan',
 ' \nIt stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, Matt Damon, and Michael Caine',
 ' \nSet in a dystopian future where humanity is struggling to survive, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for mankind',
 'Brothers Christopher and Jonathan Nolan wrote the screenplay, which had its origins in a script Jonathan developed in 2007',
 ' \nCaltech theoretical physicist and 2017 Nobel laureate in Physics[4] Kip Thorne was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar',
 ' \nCinematographer Hoyte van Hoytema shot it on 35 mm movie film in the Panavision anamorphic format and IMAX 70 mm',
 ' \nPrincipal photography began in late 2013 and took place in Alberta, Iceland, and Los Angeles',
 ' \nInterstellar uses ext

In [7]:
# get the embedding

response = co.embed(
    
    texts = texts,
    input_type = "seach_documnet",
).embeddings

embeds = np.array(response)

print(embeds.shape) # 15 -> vectors and each of them have size of 4096. 

(15, 4096)


In [11]:
# building the search index 
# it is useful for the quick search but how ?
# can retrieve close neighbours fast even when we have large number of points. 

import faiss 

dim = embeds.shape[1] # creates the dim 
index = faiss.IndexFlatL2(dim) # The residual vector is the difference between a vector 
print(index.is_trained)
index.add(np.float32(embeds))



True


In [21]:
def search(query, number_of_results=3):
    
    # 1. get the query embedding 
    
    query_embed = co.embed(texts=[query],
                           input_type="search_query").embeddings[0]
    
    # 2. retrieve the nearest results
    
    distances , similar_items_ids = index.search(np.float32([query_embed]), number_of_results )
    
    # 3. format the results 
    
    texts_np = np.array(texts) # conver the texts list to numpy for easier indexing 
    
    results = pd.DataFrame(data={'texts' : texts_np[similar_items_ids[0]],
                                 'distance' : distances[0]}) # show result in good format 
    
    # 4. print 
    
    print(f"Query : '{query}'\nnearest neighbours : ")
    
    return results
    
    
    

In [22]:
query = "how precise was the science"
results = search(query)
results

Query : 'how precise was the science'
nearest neighbours : 


,texts,distance
0,\nIt has also received praise from many astro...,12080.429688
1,\nCaltech theoretical physicist and 2017 Nobe...,12341.482422
2,\nInterstellar uses extensive practical and m...,12400.904297
